# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers
# import keras
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Embedding
# from keras.layers import SimpleRNN
# from keras.datasets import imdb
# from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17473536/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 14ms/step - loss: 0.6644 - accuracy: 0.6252 - val_loss: 0.6111 - val_accuracy: 0.6750
Epoch 2/10
782/782 [==============================] - 12s 15ms/step - loss: 0.5659 - accuracy: 0.7115 - val_loss: 0.5522 - val_accuracy: 0.7126
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.5074 - accuracy: 0.7547 - val_loss: 0.5193 - val_accuracy: 0.7368
Epoch 4/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4677 - accuracy: 0.7802 - val_loss: 0.4974 - val_accuracy: 0.7519
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4389 - accuracy: 0.7981 - val_loss: 0.4854 - val_accuracy: 0.7596
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4167 - accuracy: 0.8093 - val_loss: 0.4704 - val_accuracy: 0.7702
Epoch 7/10
782/782 [==============================] - 11s 15ms/step - loss: 0.3998 - accuracy: 0.8197 - val_loss: 0.4621 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 3ms/step - loss: 0.4494 - accuracy: 0.7861
Test score: 0.4493938386440277
Test accuracy: 0.786080002784729


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 22s 26ms/step - loss: 0.6192 - accuracy: 0.6486 - val_loss: 0.5636 - val_accuracy: 0.7123
Epoch 2/10
782/782 [==============================] - 20s 26ms/step - loss: 0.4739 - accuracy: 0.7763 - val_loss: 0.4827 - val_accuracy: 0.7708
Epoch 3/10
782/782 [==============================] - 20s 26ms/step - loss: 0.4046 - accuracy: 0.8202 - val_loss: 0.4452 - val_accuracy: 0.7844
Epoch 4/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3606 - accuracy: 0.8434 - val_loss: 0.4130 - val_accuracy: 0.8122
Epoch 5/10
782/782 [==============================] - 21s 26ms/step - loss: 0.3322 - accuracy: 0.8576 - val_loss: 0.4053 - val_accuracy: 0.8159
Epoch 6/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3117 - accuracy: 0.8684 - val_loss: 0.4369 - val_accuracy: 0.8015
Epoch 7/10
782/782 [==============================] - 20s 26ms/step - loss: 0.2948 - accuracy: 0.8784 - val_loss: 0.3804 - val_accuracy:

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 19s 22ms/step - loss: 0.6445 - accuracy: 0.6276 - val_loss: 0.5659 - val_accuracy: 0.7074
Epoch 2/10
782/782 [==============================] - 17s 22ms/step - loss: 0.5049 - accuracy: 0.7553 - val_loss: 0.4893 - val_accuracy: 0.7600
Epoch 3/10
782/782 [==============================] - 17s 21ms/step - loss: 0.4416 - accuracy: 0.7977 - val_loss: 0.4801 - val_accuracy: 0.7692
Epoch 4/10
782/782 [==============================] - 17s 21ms/step - loss: 0.4020 - accuracy: 0.8191 - val_loss: 0.4282 - val_accuracy: 0.8008
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3726 - accuracy: 0.8363 - val_loss: 0.4114 - val_accuracy: 0.8104
Epoch 6/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3511 - accuracy: 0.8477 - val_loss: 0.3941 - val_accuracy: 0.8213
Epoch 7/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3356 - accuracy: 0.8551 - val_loss: 0.3802 - val_accuracy:

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3011 - accuracy: 0.8729 - val_loss: 0.3752 - val_accuracy: 0.8321
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2957 - accuracy: 0.8759 - val_loss: 0.3652 - val_accuracy: 0.8404
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.2917 - accuracy: 0.8778 - val_loss: 0.3633 - val_accuracy: 0.8392
Epoch 4/10
782/782 [==============================] - 17s 21ms/step - loss: 0.2871 - accuracy: 0.8798 - val_loss: 0.3623 - val_accuracy: 0.8412
Epoch 5/10
782/782 [==============================] - 17s 21ms/step - loss: 0.2840 - accuracy: 0.8816 - val_loss: 0.3619 - val_accuracy: 0.8421
Epoch 6/10
782/782 [==============================] - 17s 21ms/step - loss: 0.2809 - accuracy: 0.8837 - val_loss: 0.3595 - val_accuracy: 0.8415
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2785 - accuracy: 0.8847 - val_loss: 0.3593 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation